In [1]:
from transformers import MarianMTModel, MarianTokenizer
import torch

In [2]:
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")

Downloading:   0%|          | 0.00/768k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

In [3]:
embeddings = model.get_input_embeddings()

# create ids of encoded input vectors
input_ids = tokenizer("I want to buy a car", return_tensors="pt").input_ids

# pass input_ids to encoder
encoder_hidden_states = model.base_model.encoder(input_ids, return_dict=True).last_hidden_state

In [4]:
# change the input slightly and pass to encoder
input_ids_perturbed = tokenizer("I want to buy a house", return_tensors="pt").input_ids
encoder_hidden_states_perturbed = model.base_model.encoder(input_ids_perturbed, return_dict=True).last_hidden_state

# compare shape and encoding of first vector
print(f"Length of input embeddings {embeddings(input_ids).shape[1]}. Length of encoder_hidden_states {encoder_hidden_states.shape[1]}")

# compare values of word embedding of "I" for input_ids and perturbed input_ids
print("Is encoding for `I` equal to its perturbed version?: ", torch.allclose(encoder_hidden_states[0, 0], encoder_hidden_states_perturbed[0, 0], atol=1e-3))

Length of input embeddings 7. Length of encoder_hidden_states 7
Is encoding for `I` equal to its perturbed version?:  False


In [9]:
tokenizer("I want to buy a house house", return_tensors="pt").input_ids

tensor([[  38,   38,  392,   12, 2387,   14,  869,  869,    0]])

In [12]:
emb = model.base_model.encoder(input_ids, return_dict=True)

In [14]:
emb.last_hidden_state

tensor([[[ 0.1171, -0.0477,  0.0791,  ..., -0.5256, -0.3510,  0.0203],
         [-0.1621,  0.1312, -0.1019,  ..., -0.1522, -0.0629,  0.0548],
         [-0.0190,  0.3175, -0.0063,  ...,  0.0165, -0.3745,  0.1352],
         ...,
         [ 0.2279,  0.5423,  0.1039,  ..., -0.1859, -0.2347,  0.2879],
         [-0.0718,  0.4757,  0.0434,  ..., -0.1713, -0.2366, -0.2629],
         [-0.0405,  0.1421,  0.0025,  ..., -0.1198, -0.0124, -0.0020]]],
       grad_fn=<NativeLayerNormBackward>)

In [16]:
emb.last_hidden_state.size()

torch.Size([1, 7, 512])